In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics
import collections



from sklearn.model_selection import train_test_split

#import data, add own location
train_data = pd.read_csv(r'C:\Users\User\Desktop\Credit_Score_Classification\train.csv')
train_data = pd.DataFrame(train_data)

C:\Users\User\AppData\Local\Temp\ipykernel_17104\1367173597.py:14: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(r'C:\Users\User\Desktop\Credit_Score_Classification\train.csv')


In [2]:
#preprocessing
# Drop Unimportant Columns
data1 = train_data.drop(columns=['ID','Customer_ID','Month','Name','SSN','Monthly_Inhand_Salary',
                           'Type_of_Loan','Num_of_Delayed_Payment','Credit_History_Age','Amount_invested_monthly'])
# Drop weird value
data1.drop(data1[data1['Monthly_Balance']=='__-333333333333333333333333333__'].index, inplace=True)


# Deleting (-) and (_) in Column
data1['Age'] = data1['Age'].str.replace('-','')
data1['Age'] = data1['Age'].str.replace('_','')

# Deleting (-) and (_) in Column
data1['Annual_Income'] = data1['Annual_Income'].str.replace('-','')
data1['Annual_Income'] = data1['Annual_Income'].str.replace('_','')

# Deleting (-) and (_) in Column
data1['Num_of_Loan'] = data1['Num_of_Loan'].str.replace('-','')
data1['Num_of_Loan'] = data1['Num_of_Loan'].str.replace('_','')

# Deleting (-) and (_) in Column
data1['Changed_Credit_Limit'] = data1['Changed_Credit_Limit'].str.replace('-','0')
data1['Changed_Credit_Limit'] = data1['Changed_Credit_Limit'].str.replace('_','0')

# Deleting (-) and (_) in Column
data1['Outstanding_Debt'] = data1['Outstanding_Debt'].str.replace('-','')
data1['Outstanding_Debt'] = data1['Outstanding_Debt'].str.replace('_','')

# Deleting (-) and (_) in Column
data1['Monthly_Balance'] = data1['Monthly_Balance'].str.replace('-','')
data1['Monthly_Balance'] = data1['Monthly_Balance'].str.replace('_','')

# Replacing (!@9#%8) to (Unknown) in Column
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('!@9#%8','Unknown')



data1['Age'] = data1['Age'].astype(int)
data1['Annual_Income'] = data1['Annual_Income'].astype(float)
data1['Num_of_Loan'] = data1['Num_of_Loan'].astype(int)
data1['Changed_Credit_Limit'] = data1['Changed_Credit_Limit'].astype(float)
data1['Outstanding_Debt'] = data1['Outstanding_Debt'].astype(float)
data1['Monthly_Balance'] = data1['Monthly_Balance'].astype(float)





data1['Payment_of_Min_Amount'] = data1['Payment_of_Min_Amount'].str.replace('NM','0')
data1['Payment_of_Min_Amount'] = data1['Payment_of_Min_Amount'].str.replace('No','1')
data1['Payment_of_Min_Amount'] = data1['Payment_of_Min_Amount'].str.replace('Yes','2')
data1['Payment_of_Min_Amount']=data1['Payment_of_Min_Amount'].astype(int)


data1['Age'].where((data1['Age']<=80),np.nan, inplace=True)
data1['Age'] = data1['Age'].fillna(method='ffill')


# Handling missing values with mean
data1['Monthly_Balance'] = data1['Monthly_Balance'].fillna(data1['Monthly_Balance'].mean())
data1['Num_Credit_Inquiries'] = data1['Num_Credit_Inquiries'].fillna(data1['Num_Credit_Inquiries'].mean())

# Handling missing values with ffill
data1['Occupation'] = data1['Occupation'].replace('_______', method='ffill')
data1.head()

# Handling missing values with bfill
data1['Credit_Mix'] = data1['Credit_Mix'].replace('_', method='bfill')
data1.head()


data1['Credit_Mix'] = data1['Credit_Mix'].str.replace('Bad','0')
data1['Credit_Mix'] = data1['Credit_Mix'].str.replace('Standard','1')
data1['Credit_Mix'] = data1['Credit_Mix'].str.replace('Good','2')
data1['Credit_Mix'] = data1['Credit_Mix'].astype(int)
data1.groupby(['Credit_Mix']).count()[['Occupation']]



data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('Unknown','0')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('Low_spent_Small_value_payments','1')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('Low_spent_Medium_value_payments','2')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('Low_spent_Large_value_payments','3')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('High_spent_Small_value_payments','4')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('High_spent_Medium_value_payments','5')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].str.replace('High_spent_Large_value_payments','6')
data1['Payment_Behaviour'] = data1['Payment_Behaviour'].astype(int)
data1.groupby(['Payment_Behaviour']).count()[['Occupation']]

data1['Credit_Score'] = data1['Credit_Score'].str.replace('Poor','0')
data1['Credit_Score'] = data1['Credit_Score'].str.replace('Standard','1')
data1['Credit_Score'] = data1['Credit_Score'].str.replace('Good','2')
data1['Credit_Score'] = data1['Credit_Score'].astype(int)
data1.groupby(['Credit_Score']).count()[['Occupation']]



data1 = pd.get_dummies(data1, columns=['Occupation'], prefix=['Occ_'])

data2=data1.copy()

# Capping the outliers with the extreme values in the data distribution range: Winsorization
def treat_outliers(col, df):
    lower_limit, upper_limit = df[col].quantile([0.25,0.75])
    IQR = upper_limit - lower_limit
    lower_whisker = lower_limit - 1.5 * IQR
    upper_whisker = upper_limit + 1.5 * IQR   
    return np.where(df[col]>upper_whisker,upper_whisker,np.where(df[col]<lower_whisker,lower_whisker,df[col]))

outlier_cols = ['Total_EMI_per_month','Credit_Utilization_Ratio','Num_Credit_Inquiries','Interest_Rate','Num_Credit_Card']

for col in outlier_cols:
    data1[col] = treat_outliers(col, data1)

In [3]:
#overview over selected variables
pd.options.display.float_format = '{:.2f}'.format
data1[["Monthly_Balance",  "Age", "Credit_Utilization_Ratio", "Annual_Income", "Num_of_Loan"]].describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Balance,99991.00,402.16,210.56,0.01,271.83,341.40,463.50,1602.04
Age,99991.00,33.32,10.77,14.00,24.00,33.00,42.00,56.00
Credit_Utilization_Ratio,99991.00,32.28,5.12,20.00,28.05,32.31,36.50,49.16
Annual_Income,99991.00,176426.79,1429681.87,7005.93,19456.50,37578.61,72790.92,24198062.00
Num_of_Loan,99991.00,10.76,61.79,0.00,2.00,3.00,6.00,1496.00


In [4]:
from imblearn.over_sampling import ADASYN,RandomOverSampler




#balance data for better training for underrepresented classes
def split_and_balance(df,target='Credit_Score',test_size=0.3,random_state=0,
                      augmentation=RandomOverSampler(random_state=16),remove_loan_type=False):

    if remove_loan_type:
        df=df.loc[:,:'Credit_Score']
    test=pd.DataFrame()
    nuniq_labels=df[target].nunique()

    for i,l in enumerate(df[target].unique()):
        if i ==0:
            test=df.query(f'Credit_Score=={l}').sample(n=int((df.shape[0]*test_size)//nuniq_labels)
                                                       ,random_state=random_state)
        else:
            test=pd.concat([test,df.query(f'Credit_Score=={l}')
                           .sample(n=int((df.shape[0]*test_size)//nuniq_labels),
                                   random_state=random_state)])
    train=df.drop(test.index)

    if augmentation:
        xtrain,ytrain=augmentation.fit_resample(train.drop(target,axis=1),
                                                train[target])

    else:
        xtrain,ytrain=train.drop(target,axis=1),train[target]

    xtest,ytest=test.drop(target,axis=1),test[target]

    return xtrain,xtest,ytrain,ytest


normal_columns=['Age', 'Annual_Income', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries',
       'Outstanding_Debt', 'Credit_Utilization_Ratio',
       'Total_EMI_per_month', 'Monthly_Balance']

#normalize columns, as model doesn't converge when not done
for cl in normal_columns:
    data1[cl] =(( data1[cl] - data1[cl].mean() ) / data1[cl].std())

data1.shape

(99991, 32)

In [5]:
#train/test split
X_train, X_test, y_train, y_test=split_and_balance(df=data1,test_size=0.3)


In [6]:
import numpy as np
import sklearn
from sklearn import datasets

import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn

import matplotlib.pyplot as plt

In [7]:
#convert train, test data to tensor

x, y = torch.from_numpy(X_train.to_numpy()).float(), torch.from_numpy(y_train.to_numpy()).long()

x_t, y_t = torch.from_numpy(X_test.to_numpy()).float(), torch.from_numpy(y_test.to_numpy()).long()



In [8]:
#define and train BNN-model
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=31, out_features=200),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=200, out_features=3), 
    nn.LogSoftmax(dim=1),
)


optimizer = optim.Adam(model.parameters(), lr=0.0001)




ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 1.0


n_epochs = 100  # number of epochs to run
batch_size = 512  # size of each batch
batches_per_epoch = len(X_train) // batch_size



loss_sum=0

for epoch in range(n_epochs):
    for i in range(batches_per_epoch):
        start = i * batch_size
        # take a batch
        Xbatch = x[start:start+batch_size]
        ybatch = y[start:start+batch_size]
        # forward pass
        y_pred = model(Xbatch)
        ce = ce_loss(y_pred, ybatch)
        kl = kl_loss(model)
        loss = ce + kl_weight*kl
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()




In [9]:
#für plot

from scipy import stats
import collections


bnn_models_result_raw=[]
bnn_models_result_raw2=[]
for k in range(100):
    bnn_models_result_raw.append(np.exp(model(x_t).data))
    bnn_models_result_raw2.append(np.array(np.exp(model(x_t).data)))



#append index with the highest softmax output and the data
bnn_models_result_max_index=[]
bnn_models_result_rawda=[]
for tens in bnn_models_result_raw:
    bnn_models_result_max_index.append(np.array(torch.max(tens, 1)[1]))
    max_indx=np.array(torch.max(tens, 1)[1])
    bnn_models_result_rawda.append(np.array(tens))

bnn_models_result_max_index=np.array(bnn_models_result_max_index).T
bnn_models_result_rawda=np.array(bnn_models_result_rawda).T


#list of most common indexes
bnn_mfreq = [collections.Counter(i).most_common()[0][0] for i in bnn_models_result_max_index]

coll_vals_ges=[]

#collect uncertainty threshold for every prediction
for i in range (0,len(bnn_mfreq)):
    coll_vals=[]
    for runs in range (0,100):
        coll_vals.append(bnn_models_result_raw2[runs][i][bnn_mfreq[i]])
    coll_vals_ges.append(np.median(coll_vals))


ytlist=y_t.tolist()
rejection_df=pd.DataFrame(data=zip(bnn_mfreq, coll_vals_ges, ytlist), columns=["mfreq", "med_prob", "y_true"])


X_test_thresh_org=X_test.reset_index(drop=True)
X_test_thresh=X_test.reset_index(drop=True)
X_test_thresholds=[]
y_test_thresholds=[]
y_pred_thresholds=[]
accuracies=[]
percent_classified=[]
label_distri=[]
label_distri2=[]

total_org=len(rejection_df['mfreq'])
rejection_df_org=rejection_df

#list of defined thresholds
thresholds=[0, 0.5, 0.6, 0.7, 0.8, 0.9]

for threshold in thresholds:

        #reject items with thresholds smaller than the current threshold
        for item in rejection_df.iterrows():
            if(item[1][1]<threshold):
                rejection_df.drop(index=item[0], inplace=True)
                X_test_thresh.drop(index=item[0], inplace=True)

        y_pred_thresholds.append(rejection_df["mfreq"])
        rejection_df=rejection_df.reset_index(drop=True)
        X_test_thresh=X_test_thresh.reset_index(drop=True)
        X_test_thresholds.append(X_test_thresh)
        y_test_thresholds.append(rejection_df['y_true'])
        
        


        total = len(rejection_df['mfreq'])

        #count correct classifications
        correct=0
        for i in range(0,len(rejection_df['mfreq'])):
                       if(rejection_df['mfreq'][i]==(rejection_df['y_true'][i])):
                            correct=correct+1

        #collects accuracies and classified percent for each threshold
        accuracies.append(100 * float(correct) / total)
        percent_classified.append(100*(total/total_org))
        rejection_df=rejection_df_org
        X_test_thresh=X_test_thresh_org
        
print('done')

done


In [10]:
#Gaussian Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))




Model accuracy score: 0.6553


In [11]:
#Simple NN classifier

model2 = nn.Sequential(
    nn.Linear(in_features=31, out_features=100),
    nn.ReLU(),
    nn.Linear(in_features=100, out_features=3), 
    nn.LogSoftmax(dim=1),
)


optimizer = optim.Adam(model2.parameters(), lr=0.0001)

x_t, y_t = torch.from_numpy(X_test.to_numpy()).float(), torch.from_numpy(y_test.to_numpy()).long()

x, y = torch.from_numpy(X_train.to_numpy()).float(), torch.from_numpy(y_train.to_numpy()).long()

ce_loss = nn.CrossEntropyLoss()



n_epochs = 100  
batch_size = 512  
batches_per_epoch = len(X_train) // batch_size




loss_sum=0

for epoch in range(n_epochs):
    for i in range(batches_per_epoch):
        start = i * batch_size
        # take a batch
        Xbatch = x[start:start+batch_size]
        ybatch = y[start:start+batch_size]
        # forward pass
        y_pred = model2(Xbatch)
        ce = ce_loss(y_pred, ybatch)
        loss = ce
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
        loss_sum=loss_sum+loss





In [12]:
y_pred= model2(x_t)
y_pred=np.array(torch.max(y_pred, 1)[1])
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score: 0.6776


In [13]:
#Evaluate models for different confidence thresholds
from sklearn.metrics import f1_score

BNN_acc=[]
BNN_f1=[]
DNN_acc=[]
DNN_f1=[]
GNB_acc=[]
GNB_f1=[]
for i in range(len(X_test_thresholds)):
    x_tt, y_tt = torch.from_numpy(X_test_thresholds[i].to_numpy()).float(), torch.from_numpy(y_test_thresholds[i].to_numpy()).long()
    y_pred= model2(x_tt)
    y_pred=np.array(torch.max(y_pred, 1)[1])
    DNN_acc.append(accuracy_score(y_test_thresholds[i], y_pred))
    DNN_f1.append(f1_score(y_test_thresholds[i], y_pred, average="macro"))
    y_pred=None
    
    
    BNN_acc.append(accuracy_score(y_test_thresholds[i], y_pred_thresholds[i]))
    BNN_f1.append(f1_score(y_test_thresholds[i], y_pred_thresholds[i], average="macro"))
    
    
    y_pred = gnb.predict(X_test_thresholds[i])
    GNB_acc.append(accuracy_score(y_test_thresholds[i], y_pred))
    GNB_f1.append(f1_score(y_test_thresholds[i], y_pred, average="macro"))

rejection_df=pd.DataFrame(data=zip(BNN_acc, BNN_f1,DNN_acc, DNN_f1,GNB_acc, GNB_f1, percent_classified), index=thresholds, 
                          columns=["BNN Acc.", "BNN F1","DNN Acc.", "DNN F1","GNB Acc.", "GNB F1", "share accepted"])



rejection_df.T

,0.00,0.50,0.60,0.70,0.80,0.90
BNN Acc.,0.64,0.70,0.74,0.78,0.80,0.84
BNN F1,0.62,0.56,0.55,0.57,0.59,0.59
DNN Acc.,0.68,0.71,0.74,0.78,0.80,0.84
DNN F1,0.66,0.58,0.55,0.57,0.59,0.59
GNB Acc.,0.66,0.70,0.74,0.78,0.80,0.84
GNB F1,0.64,0.58,0.55,0.57,0.59,0.59
share accepted,100.00,69.96,53.82,33.48,12.43,1.40


In [14]:
for i in y_test_thresholds:
    print(collections.Counter(i))

Counter({2: 9999, 1: 9999, 0: 9999})
Counter({0: 8091, 2: 8039, 1: 4856})
Counter({0: 6540, 2: 6387, 1: 3218})
Counter({0: 4432, 2: 3747, 1: 1865})
Counter({0: 2205, 2: 837, 1: 686})
Counter({0: 341, 1: 66, 2: 13})


In [15]:
#use unmodified data as basis for outliers

x = data2.drop('Credit_Score', axis=1) # Independet Variable
x = torch.from_numpy(x.to_numpy()).float()


max_input_np = np.zeros((8, x.shape[1]))

#make outliers
#(indexes 0,1: small, common values; 2,3:high, uncommon values; 4,5: median, mean values, 6,7: zero(nonsense values) )
#low values
for i in range(x.shape[1]):
    max_input_np[0, i] = torch.amin(x[:, i]) * 1

for i in range(x.shape[1]):
    max_input_np[1, i] = torch.amin(x[:, i]) * 0.1

    
    
for i in [6,7,10]:
    max_input_np[0, i] = torch.amax(x[:, i]) * 1    
for i in [6,7,10]:
    max_input_np[1, i] = torch.amax(x[:, i]) * 10

    
#high values   
for i in range(x.shape[1]):
    max_input_np[2, i] = torch.amax(x[:, i]) * 1    
for i in range(x.shape[1]):
    max_input_np[3, i] = torch.amax(x[:, i]) * 10
    
for i in [6,7,10]:
    max_input_np[2, i] = torch.amin(x[:, i]) * 1    
for i in [6,7,10]:
    max_input_np[3, i] = torch.amin(x[:, i]) * 0.1

#average values
for i in range(x.shape[1]):
    max_input_np[4, i] = torch.median(x[:, i])      
for i in range(x.shape[1]):
    max_input_np[5, i] = torch.mean(x[:, i])

#nonsense values 
for i in range(x.shape[1]):
    max_input_np[6, i] = 0
    
for i in range(x.shape[1]):
    max_input_np[7, i] = torch.median(x[:, i])

    
#manually set categorical attributes    
for i in [0,1,4,5]:
    max_input_np[i,0] = torch.median(x[:, 0])
    max_input_np[i,9] = 1
    max_input_np[i,12] = 2
    max_input_np[i,14] = 1
    for b in range(16,31):
        max_input_np[i,b] = 0
    max_input_np[i,23] = 1

for i in [2,3]:
    max_input_np[i,0] = torch.median(x[:, 0])
    max_input_np[i,9] = 0
    max_input_np[i,12] = 0
    max_input_np[i,14] = 0
    for b in range(16,31):
        max_input_np[i,b] = 0
    max_input_np[i,30] = 1
        
for i in [7]:
    max_input_np[i,9] = 1
    max_input_np[i,12] = 2
    max_input_np[i,14] = 1
    for b in range(16,31):
        max_input_np[i,b] = 1
        
        
pd.options.display.float_format = '{:.2f}'.format


#normalize manufactured data
max_i_df=pd.DataFrame(max_input_np, columns=data2.drop('Credit_Score', axis=1).columns)
for cl in normal_columns:
    max_i_df[cl] =(( max_i_df[cl] - data2[cl].mean() ) / data2[cl].std())

    
x_ttt = torch.from_numpy(max_i_df.to_numpy()).float()

In [16]:
#predict outliers

from scipy import stats
import collections

#predict every entry 100x
np.exp(model(x_ttt).data)
bnn_models_result_raw=[]
bnn_models_result_raw2=[]
for k in range(100):
    bnn_models_result_raw.append(np.exp(model(x_ttt).data))
    bnn_models_result_raw2.append(np.array(np.exp(model(x_ttt).data)))

#collect indexes with highest softmax
bnn_models_result_max_index=[]
bnn_models_result_rawda=[]



for tens in bnn_models_result_raw:
    bnn_models_result_max_index.append(np.array(torch.max(tens, 1)[1]))
    max_indx=np.array(torch.max(tens, 1)[1])
    bnn_models_result_rawda.append(np.array(tens))

bnn_models_result_max_index=np.array(bnn_models_result_max_index).T
bnn_models_result_rawda=np.array(bnn_models_result_rawda).T


#collect most common indexes for every entry(=classification prediction)
bnn_mfreq = [collections.Counter(i).most_common()[0][0] for i in bnn_models_result_max_index]


#collect median softmax values
coll_vals_ges=[]

for i in range (0,len(bnn_mfreq)):
    coll_vals=[]
    for runs in range (0,100):
        coll_vals.append(bnn_models_result_raw2[runs][i][bnn_mfreq[i]])
    coll_vals_ges.append(np.median(coll_vals))



    

#create rejection_df for outlier values
ytlist=y_t.tolist()
rejection_df=pd.DataFrame(data=zip(bnn_mfreq, coll_vals_ges), columns=["mfreq", "med_prob"], 
                          index=["min", "min*10","max", "max*10","median", "mean","zeros", "median+nons"])


total_org=len(rejection_df['mfreq'])
rejection_df_org=rejection_df


        
rejection_df = rejection_df.rename(columns={"mfreq":"prediction", "med_prob" :"median probability"})
rejection_df['prediction'].replace(2, "'Good'", inplace=True)
rejection_df['prediction'].replace(1, "'Standard'", inplace=True)
rejection_df['prediction'].replace(0, "'Poor'", inplace=True)

        
               
    
rejection_df.T

,min,min*10,max,max*10,median,mean,zeros,median+nons
prediction,'Poor','Poor','Poor','Poor','Standard','Poor','Standard','Standard'
median probability,0.59,1.00,1.00,1.00,0.40,0.42,0.32,0.39
